# Art Classifer - Fast.ai implementation

#### Description

The following script fine-tunes a deep convolutional net (specifically, ResNet 34) to classify whether an input image came from two sets of art images that I defined. The first set, "abstract" consists of a set of images of Abstract Minimalist art by Frank Stella, Ellsworth Kelly, Sol LeWitt, Piet Mondrian, Sophie Taeuber-Arp, Josef Albers, Kazimir Malevich, and Hilma af Klint. The second set, "surreal" consists of a set of images of Surrealist art (primarily figurative) by Yves Tanguy, Salvador Dali, Rene Magritte, Max Ernst, Giorgio de Chirico, and Remedios Varo.

To do so, I scraped about 1000 images of works (~500 abstract and ~500 surreal), resized them to size 224x224 .jpg's, and placed them in labeled folders. Here, I simply use them to build a data loader for `fastai`, then use its `vision_learner` to fine-tune the pretrained ResNet 34 model. This uses an 80-20 split for the training and validation sets. I then "test" the model on 9 exemplary images. 3 were top results from searching "abstract art", 3 from searching "surreal art," and the other 3 were chosen to be intentionally ambiguous.

#### Requirements

- `numpy`
- `matplotlib`
- `pandas`
- `fastai`

In [ ]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from fastai.vision.all import *

In [ ]:
IMAGE_SIDELENGTH = 224

In [ ]:
basePath = Path('/home/mikejseay/Data/Art/')

artClasses = ('abstract', 'surreal')

paths = []
labels = []

for artClass in artClasses:
    classPath = basePath/artClass
    subDirs = glob.glob(str(classPath) + '/*')
    for subDir in subDirs:
        imagePaths = glob.glob(subDir + '/*.jpg')
        for imagePath in imagePaths:
            paths.append(imagePath)
            labels.append(artClass)

imageFileDF = pd.DataFrame.from_dict({'path': paths, 'label': labels})
dls = ImageDataLoaders.from_df(imageFileDF, path='/', item_tfms=Resize(IMAGE_SIDELENGTH))

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

In [ ]:
basePath = Path('/home/mikejseay/Data/TestArt/')

imagePaths = glob.glob(str(basePath) + '/*.jpg')

f, ax = plt.subplots(3, 3, figsize=(16, 12))
ax = ax.ravel()

for imageInd, imagePath in enumerate(imagePaths):
    img = PILImage.create(imagePath)
    predicted_class, _, probs = learn.predict(img)
    displayProbs = np.max(probs.cpu().detach().numpy())
    descriptString = '{} ({:.2%})'.format(predicted_class, displayProbs)
    ax[imageInd].imshow(np.asarray(img))
    ax[imageInd].get_xaxis().set_visible(False)
    ax[imageInd].get_yaxis().set_visible(False)
    ax[imageInd].set(title=descriptString)